In [7]:
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

def collect_melon(chartdate):
    headers = {
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                       '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5'
    }
    age_url = "https://www.melon.com/chart/age/list.htm"
    params = {
        'idx': '1',
        'chartType': 'YE',     # 년도별로
        'chartGenre': 'KPOP',  # 한국가요
        'chartDate': chartdate,   # 검색연도
        'moved': 'Y',
    }
    response = requests.get(age_url, params=params, headers=headers)
    if response.status_code != 200:
        print(f"Request failed with status code {response.status_code}. Potentially blocked.")
        return
    if "Access Denied" in response.text or "blocked" in response.text:
        print("Access appears to be blocked.")
        return
    response = requests.get(age_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    song_list = soup.select('.lst50')

    #데이터 담을 리스트 생성
    data = []
    # 엑셀 파일을 생성할 경로 설정
    excel_file_path = f"./{chartdate}.xlsx"  
    for i, meta in enumerate(song_list, 1):
        # 순위, 제목
        rank = i
        print(params['chartDate'], "_", rank)
        try:
            title = meta.select('a[href*=playSong]')[0].text
        except:
            title = meta.select('.wrap_song_info .ellipsis')[0].text
        title = title.strip()

        # 노래 데이터 URL의 HTML 추출
        song_id_html = str(meta.select('a[onclick*=SongDetail]'))
        matched = re.search(r"\'(\d+)\'", song_id_html)
        song_id = matched.group(1)
        front_url = 'https://www.melon.com/song/detail.htm?songId='
        song_url = front_url + song_id

        response = requests.get(song_url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"Request failed with status code {response.status_code}. Potentially blocked.")
            return
        if "Access Denied" in response.text or "blocked" in response.text:
            print("Access appears to be blocked.")
            return
        
        # 가수, 장르 등 노래 세부 정보 추출
        response = requests.get(song_url, params=params, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        singer_html = soup.select('.wrap_info .artist a')

        # 가수
        singer_s = []
        if len(singer_html) != 0:
            for html in singer_html:
                singer_s.append(html['title'])
        else:
            # URL 없는 Various Artists용
            singer_html = str(soup.select('.wrap_info .artist')[0])
            singer_html = singer_html.replace('\t', '').replace('\r', '').split('\n')
            singer_html = ''.join(singer_html)
            matched = re.search(r">(.*)<", singer_html)
            singer_s.append(matched.group(1))

        # 가수가 여러명일 때 하나의 string으로 표현
        singer_s = ', '.join(singer_s)

        # 장르
        try:
            song_genre_html = str(soup.select('.list dd')[2])
            matched = re.search(r">(.*)<", song_genre_html)
            song_genre = matched.group(1)
        except:
            song_genre = "없음"

        # 가사 추출
        try:
            lyric_html = str(soup.select('.section_lyric .wrap_lyric .lyric')[0])
            lyric_html = lyric_html.replace('\t', '').replace('\r', '').split('\n')
            lyric_html = ''.join(lyric_html)

            matched = re.search(r"-->(.*)<br/>", lyric_html)
            lyric = matched.group(1).strip()
            lyric = lyric.replace('<br/>', '\n')

            # 가사 앞뒤 빈칸 제거
            lyric_list = []
            for line in lyric.split('\n'):
                lyric_list.append(line.strip())
            lyric = ('\n').join(lyric_list)

        except:
            lyric = "없음"

        # 데이터 리스트에 추가
        data.append([params['chartDate'], singer_s, title, song_genre, lyric,song_url])

        # IP 차단 방지용
        sleep(1)

    # pandas DataFrame 생성 및 Excel 파일로 저장
    df = pd.DataFrame(data, columns=['Year', 'Singer', 'Title', 'Genre', 'Lyric','URL'])
    df.to_excel(excel_file_path, index=False)

for i in range(2015,2024): #원하는 년도 설정(23년까지)
    collect_melon(i)

2015 _ 1
2015 _ 2
2015 _ 3
2015 _ 4
2015 _ 5
2015 _ 6
2015 _ 7
2015 _ 8
2015 _ 9
2015 _ 10
2015 _ 11
2015 _ 12
2015 _ 13
2015 _ 14
2015 _ 15
2015 _ 16
2015 _ 17
2015 _ 18
2015 _ 19
2015 _ 20
2015 _ 21
2015 _ 22
2015 _ 23
2015 _ 24
2015 _ 25
2015 _ 26
2015 _ 27
2015 _ 28
2015 _ 29
2015 _ 30
2015 _ 31
2015 _ 32
2015 _ 33
2015 _ 34
2015 _ 35
2015 _ 36
2015 _ 37
2015 _ 38
2015 _ 39
2015 _ 40
2015 _ 41
2015 _ 42
2015 _ 43
2015 _ 44
2015 _ 45
2015 _ 46
2015 _ 47
2015 _ 48
2015 _ 49
2015 _ 50
2016 _ 1
2016 _ 2
2016 _ 3
2016 _ 4
2016 _ 5
2016 _ 6
2016 _ 7
2016 _ 8
2016 _ 9
2016 _ 10
2016 _ 11
2016 _ 12
2016 _ 13
2016 _ 14
2016 _ 15
2016 _ 16
2016 _ 17
2016 _ 18
2016 _ 19
2016 _ 20
2016 _ 21
2016 _ 22
2016 _ 23
2016 _ 24
2016 _ 25
2016 _ 26
2016 _ 27
2016 _ 28
2016 _ 29
2016 _ 30
2016 _ 31
2016 _ 32
2016 _ 33
2016 _ 34
2016 _ 35
2016 _ 36
2016 _ 37
2016 _ 38
2016 _ 39
2016 _ 40
2016 _ 41
2016 _ 42
2016 _ 43
2016 _ 44
2016 _ 45
2016 _ 46
2016 _ 47
2016 _ 48
2016 _ 49
2016 _ 50
2017 _ 1
2017 _ 2


In [8]:
import glob

# 파일 경로 설정 (확장자를 가진 모든 엑셀 파일 불러오기)
file_paths = glob.glob("C:/Users/lsc/Desktop/melon/raw1/*.xlsx")

# 모든 파일을 데이터프레임으로 읽어서 리스트에 저장
dfs = [pd.read_excel(file) for file in file_paths]

# 모든 데이터프레임을 하나로 합치기
merged_df = pd.concat(dfs, ignore_index=True)

# 합쳐진 데이터를 하나의 엑셀 파일로 저장
merged_df.to_excel("merged_file.xlsx", index=False)

In [9]:
import numpy as np
np.unique(merged_df['Year'],return_counts=True)

(array([1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980,
        1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
        1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
        2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
        2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
       dtype=int64),
 array([26, 32, 44, 39, 41, 38, 39, 35, 48, 43, 30, 40, 39, 27, 50, 50, 50,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,
        50, 50, 50], dtype=int64))